# Spam-Ham
Dataset obtained from: https://archive.ics.uci.edu/ml/datasets/sms+spam+collection

It contains over 5000 SMS labeled messages that have been collected for mobile phone spam research.
This is a text classification for the mentioned dataset, using Natural Language Processing

In [2]:
import sys
import sklearn
import nltk
import pandas
import numpy

print('Python Version: {}'.format(sys.version))
print('Sci-Kit-learn Version: {}'.format(sklearn.__version__))
print('NLTK Version: {}'.format(nltk.__version__))
print('Pandas Version: {}'.format(pandas.__version__))
print('NumPy Version: {}'.format(numpy.__version__))

Python Version: 3.9.5 (tags/v3.9.5:0a7dcbd, May  3 2021, 17:27:52) [MSC v.1928 64 bit (AMD64)]
Sci-Kit-learn Version: 0.24.2
NLTK Version: 3.6.2
Pandas Version: 1.2.4
NumPy Version: 1.19.5


In [3]:
import pandas as pd
import numpy as np

# load the dataset of the SMS messages
df = pd.read_table('SMSSpamCollection', header = None, encoding = 'utf-8')

In [4]:
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       5572 non-null   object
 1   1       5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
None
      0                                                  1
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [10]:
classes = df[0]
print(classes.value_counts())

ham     4825
spam     747
Name: 0, dtype: int64


In [11]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
Y = encoder.fit_transform(classes)
print(Y[:10])

[0 0 1 0 0 1 0 0 1 1]


In [12]:
text_messages = df[1]
print(text_messages[:10])

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
5    FreeMsg Hey there darling it's been 3 week's n...
6    Even my brother is not like to speak with me. ...
7    As per your request 'Melle Melle (Oru Minnamin...
8    WINNER!! As a valued network customer you have...
9    Had your mobile 11 months or more? U R entitle...
Name: 1, dtype: object


In [15]:
# Replacing email addresses with 'emailaddress'
processed = text_messages.str.replace(r'^.+@[^\.].*\.[a-z]{2,}$','emailaddress')

# Replacing URLs with 'webaddress'
processed = processed.str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$', 'webaddress')

# Replace money symbols with 'moneysymb'
processed = processed.str.replace(r'£|\$', 'moneysymb')

# Replace 10 digit phone numbers  with 'phonenumber'
processed = processed.str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$', 'phonenumbr')

# Replace numbers with 'numbr'
processed = processed.str.replace(r'\d+(\.\d+)?', 'numbr')

# Remove punctuation
processed = processed.str.replace(r'[^\w\d\s]', ' ')

# Replace whitespace between terms with a single space
processed = processed.str.replace(r'\s+', ' ')

# Remove leading and trailing whitespace
processed = processed.str.replace(r'^\s+|\s+?$', '')

<ipython-input-15-95a32f5ec9b4>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  processed = text_messages.str.replace(r'^.+@[^\.].*\.[a-z]{2,}$','emailaddress')
<ipython-input-15-95a32f5ec9b4>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  processed = processed.str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$', 'webaddress')
<ipython-input-15-95a32f5ec9b4>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  processed = processed.str.replace(r'£|\$', 'moneysymb')
<ipython-input-15-95a32f5ec9b4>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  processed = processed.str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$', 'phonenumbr')
<ipython-input-15-95a32f5ec9b4>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  processed =

In [16]:
# Changing everything to lower case

processed = processed.str.lower()
print(processed)

0       go until jurong point crazy available only in ...
1                                 ok lar joking wif u oni
2       free entry in numbr a wkly comp to win fa cup ...
3             u dun say so early hor u c already then say
4       nah i don t think he goes to usf he lives arou...
                              ...                        
5567    this is the numbrnd time we have tried numbr c...
5568                  will ü b going to esplanade fr home
5569    pity was in mood for that so any other suggest...
5570    the guy did some bitching but i acted like i d...
5571                            rofl its true to its name
Name: 1, Length: 5572, dtype: object


In [17]:
from nltk.corpus import stopwords

# Removing the stop words from text messages

stop_words = set(stopwords.words('english'))

processed = processed.apply(lambda x: ' '.join(term for term in x.split() if term not in 
                                               stop_words))

In [19]:
# Removing the word stems (ex: -ing, -ed etc)

ps = nltk.PorterStemmer()

processed = processed.apply(lambda x: ' '.join(ps.stem(term) for term in x.split()))

In [20]:
# generating the features

from nltk.tokenize import word_tokenize
all_words = []

for message in processed:
    words = word_tokenize(message)
    for w in words:
        all_words.append(w)
        
all_words = nltk.FreqDist(all_words)

In [22]:
# Printing the 15 most common words

print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(15)))

Number of words: 6579
Most common words: [('numbr', 2648), ('u', 1207), ('call', 674), ('go', 456), ('get', 451), ('ur', 391), ('gt', 318), ('lt', 316), ('come', 304), ('moneysymbnumbr', 303), ('ok', 293), ('free', 284), ('day', 276), ('know', 275), ('love', 266)]


In [24]:
word_features = list(all_words.keys())[:1500] # using 1500 most common words as features

In [29]:
# find_features function will determine which of the 1500 word features are contained in the review

def find_features(message):
    words = word_tokenize(message)
    features = {}
    for word in word_features:
        features[word] = (word in words)

    return features

# Lets see an example!
features = find_features(processed[0])
for key, value in features.items():
    if value == True:
        print (key)

go
jurong
point
crazi
avail
bugi
n
great
world
la
e
buffet
cine
got
amor
wat


In [31]:
messages = list(zip(processed, Y))
seed = 1
np.random.seed = seed
np.random.shuffle(messages)

featuresets = [(find_features(text), label) for (text, label) in messages]

In [32]:
from sklearn import model_selection

training, testing = model_selection.train_test_split(featuresets, test_size = 0.25, random_state = seed )


In [33]:
print(len(training))
print(len(testing))

4179
1393


In [34]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.svm import SVC

model = SklearnClassifier(SVC(kernel = 'linear'))
model.train(training)
accuracy = nltk.classify.accuracy(model, testing)*100

print("SVC Accuracy: {}".format(accuracy))

SVC Accuracy: 98.85139985642498


In [36]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

names = ["K Nearest Neighbors",
        "Decision Tree",
        "Random Forest",
        "Logistic Regression",
        "SGD Classifier",
        "Naive Bayes",
        "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = zip(names, classifiers)

for name, model in models:
    nltk_model = SklearnClassifier(model)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model, testing)*100
    print("{} Accuracy: {}".format(name, accuracy))

K Nearest Neighbors Accuracy: 94.75951184493898
Decision Tree Accuracy: 97.34386216798278
Random Forest Accuracy: 98.7078248384781
Logistic Regression Accuracy: 98.85139985642498
SGD Classifier Accuracy: 98.63603732950466
Naive Bayes Accuracy: 98.85139985642498


In [38]:
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))

nltk_ensemble = SklearnClassifier(VotingClassifier(estimators = models, voting = 'hard', n_jobs = -1))
nltk_ensemble.train(training)
accuracy = nltk.classify.accuracy(nltk_model, testing)*100
print("Voting Classifier accuracy: {}".format(accuracy))

Voting Classifier accuracy: 98.85139985642498


In [39]:
# Making class label predictions for testing set
txt_features, labels = list(zip(*testing))
prediction = nltk_ensemble.classify_many(txt_features)


In [40]:
print(classification_report(labels, prediction))


pd.DataFrame(
    confusion_matrix(labels, prediction),
    index =[['actual','actual'],['ham','ham']],
    columns = [['predicted','predicted'], ['ham','spam']])
      

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1215
           1       0.99      0.93      0.96       178

    accuracy                           0.99      1393
   macro avg       0.99      0.97      0.98      1393
weighted avg       0.99      0.99      0.99      1393



predicted     
                 ham spam
actual ham      1213    2
       ham        12  166